# 01 - Data Preparation: US–Venezuela Shock 2026

Tujuan:
- Mengumpulkan dan membersihkan data harga untuk:
  - Minyak: Brent, WTI
  - EM: EEM (Emerging Markets ETF), ILF (Latin America 40 ETF)
  - Crypto: BTC-USD, ETH-USD
- Menghitung log returns harian
- Menyimpan dataset yang siap dipakai untuk event study dan model lain

Periode data:
- Dari 1 Oktober 2025 sampai 31 Januari 2026

Event utama:
- 3 Januari 2026: Serangan udara AS ke Venezuela dan penangkapan Nicolás Maduro (Operation "Absolute Resolve").

In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

In [7]:
start_date = "2025-10-01"
end_date = "2026-01-31"

tickers = {
    "Brent": "BZ=F",     # Brent front-month
    "WTI": "CL=F",       # WTI front-month
    "EEM": "EEM",        # EM equity ETF
    "ILF": "ILF",        # Latin America equity ETF
    "BTC": "BTC-USD",
    "ETH": "ETH-USD",
    "SP500": "^GSPC"     # Market index sebagai proxy pasar global
}

In [9]:
price_data = {}

for name, ticker in tickers.items():
    # Download with progress disabled to avoid clutter
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    
    # Handle MultiIndex columns (yfinance new default)
    if isinstance(data.columns, pd.MultiIndex):
        try:
            data = data["Adj Close"]
        except KeyError:
            # Fallback to Close if Adj Close is missing
            data = data["Close"]
    elif "Adj Close" in data.columns:
        data = data["Adj Close"]
    else:
        data = data["Close"]
    
    # Ensure it is a DataFrame and rename properly
    if isinstance(data, pd.Series):
        data = data.to_frame()
        
    data.columns = [name]
    price_data[name] = data

# Gabungkan ke satu dataframe berdasarkan tanggal
prices = pd.concat(price_data.values(), axis=1)
prices.columns = price_data.keys()

# Cek head
prices.head()

,Brent,WTI,EEM,ILF,BTC,ETH,SP500
Date,,,,,,,
2025-10-01,65.349998,61.779999,53.060669,27.779501,118648.929688,4351.112305,6711.200195
2025-10-02,64.110001,60.480000,53.307140,27.642797,120681.257812,4487.923828,6715.350098
2025-10-03,64.529999,60.880001,53.464882,27.701385,122266.531250,4514.870605,6715.790039
2025-10-04,NaN,NaN,NaN,NaN,122425.429688,4489.197266,NaN
2025-10-05,NaN,NaN,NaN,NaN,123513.476562,4515.422852,NaN


# Cleaning data

In [10]:
# Cek missing values per kolom
print(prices.isna().sum())

# Forward-fill lalu backward-fill untuk handle hari libur kecil
prices = prices.ffill().bfill()

# Simpan harga ke CSV
prices.to_csv("prices_raw.csv", index=True)

Brent    31
WTI      31
EEM      31
ILF      31
BTC       0
ETH       0
SP500    31
dtype: int64


# Hitung log harian

In [11]:
returns = np.log(prices / prices.shift(1))
returns = returns.dropna()
returns.head()

,Brent,WTI,EEM,ILF,BTC,ETH,SP500
Date,,,,,,,
2025-10-02,-0.019157,-0.021267,0.004634,-0.004933,0.016984,0.030959,0.000618
2025-10-03,0.006530,0.006592,0.002955,0.002117,0.013051,0.005986,0.000066
2025-10-04,0.000000,0.000000,0.000000,0.000000,0.001299,-0.005703,0.000000
2025-10-05,0.000000,0.000000,0.000000,0.000000,0.008848,0.005825,0.000000
2025-10-06,0.014462,0.013217,0.004599,-0.005656,0.009982,0.037458,0.003640


In [12]:
returns.to_csv("returns_master.csv", index=True)
